In [1]:
import os
import os.path
import sys
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

module_path = os.path.join(os.path.abspath(os.path.join('..')), 'pymods')
if module_path not in sys.path:
    sys.path.append(module_path)
from plotutils import plot_3d_z_indices

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import nibabel as nib


In [3]:
base_filename = "../data/output/S264294/S264294_DTI_gated_20150508093306_12"
fdwi = base_filename + "_crop.nii.gz"
fbval = base_filename + ".bval"
fbvec = base_filename + ".bvec"

In [4]:
img = nib.load(fdwi)
data = img.get_data()

In [5]:
data.shape

(40, 45, 12, 65)

In [6]:
from dipy.io import read_bvals_bvecs
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)

In [21]:
from dipy.core.gradients import gradient_table
gtab = gradient_table(bvals, bvecs)
b_value = gtab.bvals[1]

In [76]:
from dipy.reconst.dti import fractional_anisotropy, mean_diffusivity

def print_tensor_metrics(D):
    eigval = np.linalg.eigvalsh(D)
    print("Eigval = ", eigval)
    print("FA = %.2f" % fractional_anisotropy(eigval))
    print("MD (x1000) = %.2f" % (mean_diffusivity(eigval) * 1000.))


def create_LDLT_matrix(lower, diag, scale = 1.):
    """Return D = LDL^T / scale """
    return (lower @ diag @ lower.transpose() *scale)

In [77]:
# Create a tensor that approximates some tissue
l_left = np.array([
    [ 1,  0, 0],
    [.1,  1, 0],
    [.2, .3, 1]])
diag_left = np.array([
    [ 1.8,  0, 0],
    [ 0,  1, 0],
    [ 0, 0,  0.2]])

D_left = create_LDLT_matrix(l_left, diag_left, scale=1./b_value)
print(D_left)
print_tensor_metrics(D_left)

[[0.0018   0.00018  0.00036 ]
 [0.00018  0.001018 0.000336]
 [0.00036  0.000336 0.000362]]
Eigval =  [0.00017561 0.0010477  0.00195669]
FA = 0.69
MD (x1000) = 1.06


In [79]:
# Create a tensor that approximates water
l_right = np.array([
    [ 1,  0, 0],
    [ 0.1,  1, 0],
    [ 0, 0.1, 1]])
diag_right = np.array([
    [ 3,  0, 0],
    [ 0,  3, 0],
    [ 0,  0, 3]])
D_right = create_LDLT_matrix(l_right, diag_right, scale=1./b_value)
print(D_right)
print_tensor_metrics(D_right)

[[0.003   0.0003  0.     ]
 [0.0003  0.00303 0.0003 ]
 [0.      0.0003  0.00303]]
Eigval =  [0.0025979  0.00301502 0.00344709]
FA = 0.14
MD (x1000) = 3.02


In [337]:
lower_error_sigma = 0.05
diag_error_sigma = 0.1 

def randomize_lower_matrix(lower, sigma = lower_error_sigma):
    ret = lower.copy()
    ret[0,1] += np.random.normal(scale=sigma)
    ret[0,2] += np.random.normal(scale=sigma)
    ret[1,2] += np.random.normal(scale=sigma)
    return(ret)

def randomize_diagonal_matrix(diag, sigma = diag_error_sigma):
    ret = diag.copy()
    ret[0,0] += np.random.normal(scale=sigma)
    ret[1,1] += np.random.normal(scale=sigma)
    ret[2,2] += np.random.normal(scale=sigma)
    return(ret)

def randomize_d(l, diag):
    return(create_LDLT_matrix(randomize_lower_matrix(l), 
                         randomize_diagonal_matrix(diag), 
                          scale = 1 /b_value))

print_tensor_metrics(randomize_d(l_left, diag_left))

print_tensor_metrics(randomize_d(l_right, diag_right))


Eigval =  [0.00016223 0.00095035 0.00200778]
FA = 0.72
MD (x1000) = 1.04
Eigval =  [0.0026922  0.00288899 0.0034927 ]
FA = 0.14
MD (x1000) = 3.02


In [339]:
qk = gtab.bvecs[~gtab.b0s_mask]
qk.shape

(64, 3)

In [368]:
data.max()

2216

In [388]:
data.dtype

dtype('<i2')

In [386]:
output_data = data.copy()

In [413]:
def create_qTDq_matrix(qk, D):
    ret = qk[:, 0] * qk[:, 0] * D[0,0] + \
        qk[:, 1] * qk[:, 1] * D[1,1] + \
        qk[:, 2] * qk[:, 2] * D[2,2] + \
        qk[:, 0] * qk[:, 1] * D[0, 1] * 2 + \
        qk[:, 0] * qk[:, 2] * D[0, 2] * 2 + \
        qk[:, 1] * qk[:, 2] * D[1, 2] * 2
    return (ret)

In [406]:
output_data.shape 

(40, 45, 12, 65)

In [411]:
20 > output_data.shape[0] / 2

False

In [415]:
for idx in np.ndindex(output_data.shape[:-1]):
    D = None
    if idx[0] > output_data.shape[0]/2:
        D = randomize_d(l_left, diag_left)
    else:
        D = randomize_d(l_right, diag_right)
    ret = create_qTDq_matrix(qk, D)
    attn = np.exp(-b_value * ret)
    scaled_attn = output_data[idx + (0,)] * attn 
    output_data[idx + (slice(1, None, None),)] = scaled_attn.astype(output_data.dtype)    

In [416]:
syn_file = base_filename + "_syn.nii.gz"
nib.save(nib.Nifti1Image(output_data, img.affine), syn_file)